## 02.02 Obtendo a base de dados

In [1]:
import pandas as pd

dados = pd.read_csv('data/creditcard.csv')
dados.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## 02.03 Analisando a base de dados

In [2]:
print(dados.isna().sum())

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


In [3]:
n_trasacoes = dados['Class'].count()
n_fraudes = dados['Class'].sum()
n_normais = n_trasacoes - n_fraudes
fraudes_porc = n_fraudes / n_trasacoes
normais_porc = n_normais / n_trasacoes

print("Número de transações: ", n_trasacoes)
print("Número de fraudes: ", n_fraudes, "%.2f" %(fraudes_porc * 100))
print("Número de transações normais: ", n_normais, "%.2f" %(normais_porc * 100))


Número de transações:  284807
Número de fraudes:  492 0.17
Número de transações normais:  284315 99.83


O resultado mostra que os dados são desbalanceados.  Há uma alta porcentagem de transações normais e uma pequena porcentagem de transações que são fraudes.

## 02.05 Criando uma Árvore de Decisão com scikit-learn

In [4]:
X = dados.drop('Class', axis=1).values
y = dados['Class'].values

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

validador = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)

for treino_id, teste_id in validador.split(X, y):
    X_train, X_test = X[treino_id], X[teste_id]
    y_train, y_test = y[treino_id], y[teste_id]

In [6]:
from sklearn import tree

classificador_arvore_decisao = tree.DecisionTreeClassifier()
arvore = classificador_arvore_decisao.fit(X_train, y_train)
y_pred = arvore.predict(X_test)

## 02.06 Gerando uma imagem da árvore

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

def validador(X, y):
    validador = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)

    for treino_id, teste_id in validador.split(X, y):
        X_train, X_test = X[treino_id], X[teste_id]
        y_train, y_test = y[treino_id], y[teste_id]
    
    return X_train, X_test, y_train, y_test

In [8]:
%%time
from sklearn import tree

def executar_classificador(classificador, X_train, X_test, y_train):
    arvore = classificador_arvore_decisao.fit(X_train, y_train)
    y_pred = arvore.predict(X_test)

    return y_pred

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 13.6 µs


In [9]:
X = dados.drop('Class', axis=1).values
y = dados['Class'].values

X_train, X_test, y_train, y_test = validador(X, y)

classificador_arvore_decisao = tree.DecisionTreeClassifier()
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, X_train, X_test, y_train)

## 02.07 Visualizando os nós e atributos

In [10]:
import matplotlib.pyplot as plt

def salvar_arvore(classificador, nome):
    plt.figure(figsize=(200, 100))
    tree.plot_tree(classificador, filled=True, fontsize=14)
    plt.savefig(nome)
    plt.close

In [11]:
# execução do validador
X = dados.drop('Class', axis=1).values
y = dados['Class'].values

X_train, X_test, y_train, y_test = validador(X, y)


In [12]:
# execução do classificador DecisionTreeClassifier

classificador_arvore_decisao = tree.DecisionTreeClassifier()
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, X_train, X_test, y_train)

In [13]:
# criação da figura da árvore de decisão

salvar_arvore(classificador_arvore_decisao, "arvore_decisao1.png")

## 03.02 Medindo a Acurácia da Árvore

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

def validar_arvore(y_pred, y_test):
    print(accuracy_score(y_pred, y_test))
    print(confusion_matrix(y_pred, y_test))

In [17]:
# validação da árvore de decisão

validar_arvore(y_pred_arvore_decisao, y_test)

0.9990871107053826
[[28421    15]
 [   11    34]]
